In [1]:
%load_ext autoreload
%autoreload 2

import os
import time
import datetime
import glob
import scanpy as sc
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scprep
import graphtools as gt
import phate
from scipy import sparse
from scipy.stats import zscore 
from scipy.stats import gaussian_kde

from geomloss import SamplesLoss

# import torch 
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim

import warnings
import sys
sys.path.append('/home/ngr4/project/scripts/')

plt.rc('font', size = 8)
plt.rc('font', family='sans serif')
plt.rcParams['pdf.fonttype']=42
plt.rcParams['ps.fonttype']=42
plt.rcParams['legend.frameon']=False
sns.set_style("ticks")

import os
import time
import datetime
import glob
import scanpy as sc
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scprep
import graphtools as gt
import phate
from scipy import sparse
from scipy.stats import zscore 
from scipy.stats import gaussian_kde

# from geomloss import SamplesLoss

# import torch 
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim

import warnings
import sys

import os
import sys
from pathlib import Path

import ot
import ot.plot
import seaborn as sns

sys.path.append('..')
from single_cell_analysis import datasets

plt.rc('font', size = 8)
plt.rc('font', family='sans serif')
plt.rcParams['pdf.fonttype']=42
plt.rcParams['ps.fonttype']=42
plt.rcParams['legend.frameon']=False
sns.set_style("ticks")

KeyboardInterrupt: 

In [ ]:
dfp = '/gpfs/ycga/sequencers/pacbio/gw92/10x/data_sent/maa252/091020/*/filtered_feature_bc_matrix/'
data_files = glob.glob(dfp)
sample_suffix = '_MON_crispr'

pfp = '/home/myn2/single-cell-analysis/results'
pdfp = '/home/myn2/single-cell-analysis/results'

if True:
    # first load
    adatas = {}
    for i, file in enumerate(data_files):
        if i==0:
            adata = sc.read_10x_mtx(file)
            batch_key = file.split('/filtered_')[0].split('/')[-1].split(sample_suffix)[0]
            adata.var_names_make_unique()
        else:
            adatas[file.split('/filtered_')[0].split('/')[-1].split(sample_suffix)[0]] = sc.read_10x_mtx(file)
            adatas[file.split('/filtered_')[0].split('/')[-1].split(sample_suffix)[0]].var_names_make_unique()

    adata = adata.concatenate(*adatas.values(), batch_categories=[batch_key]+list(adatas.keys()))
    del adatas
    
    sc.pp.calculate_qc_metrics(adata,inplace=True)
    mito_genes = adata.var_names.str.startswith('MT-')
    adata.obs['pmito'] = np.sum(adata[:, mito_genes].X, axis=1) / np.sum(adata.X, axis=1)
    print('Ncells=%d have >10percent mt expression' % np.sum(adata.obs['pmito']>0.1))
    print('Ncells=%d have <200 genes expressed' % np.sum(adata.obs['n_genes_by_counts']<200))
    print('Ngenes=%d have <3 genes expressed' % np.sum(adata.var['n_cells_by_counts'] < 3))
    sc.pp.filter_cells(adata, min_genes=200)
    sc.pp.filter_genes(adata, min_cells=3) # filtering cells gets rid of some genes of interest
    adata.raw = adata
    if False:
        # filter 
        adata = adata[adata.obs.pmito <= 0.1, :]
    sc.pp.normalize_total(adata)
    sc.pp.sqrt(adata)
    sc.tl.pca(adata, n_comps=100)
    
    if True :
        # save
        adata.write(os.path.join(pdfp,'crispr_shallow.h5ad'))
        print('\n... saved @'+datetime.datetime.now().strftime('%y%m%d.%H:%M:%S'))
        
else:
    adata = sc.read_h5ad(os.path.join(pdfp,'crispr_shallow.h5ad'))

In [ ]:
sc.pl.pca(adata, color=['SARS-CoV-2', 'batch'])

In [ ]:
adata.obs

In [ ]:
guide_mappings = glob.glob('/gpfs/ycga/sequencers/pacbio/gw92/10x/data_sent/maa252/091020/*/crispr_analysis/protospacer_calls_per_cell.csv')
guide_mappings

In [ ]:
mock_map = pd.read_csv('/gpfs/ycga/sequencers/pacbio/gw92/10x/data_sent/maa252/091020/MOCK_MON_crispr/crispr_analysis/protospacer_calls_per_cell.csv')
sars_map = pd.read_csv('/gpfs/ycga/sequencers/pacbio/gw92/10x/data_sent/maa252/091020/SARS2_MON_crispr/crispr_analysis/protospacer_calls_per_cell.csv')

In [ ]:
p = sns.distplot(mock_map['num_features'])
p.set_title('Number of guides detected (MOCK)')

In [ ]:
p = sns.distplot(sars_map['num_features'])
p.set_title('Number of guides detected (2dpi)')

In [ ]:
print(adata)
adata.obs.groupby('batch').count()

In [ ]:
sample = 'Mock'
mock = adata[adata.obs['batch']==sample.upper(), :]

# Generate fake data
x = mock.obs['n_genes_by_counts'].to_numpy()
y = mock.obs['total_counts'].to_numpy()

# Calculate the point density
xy = np.vstack([x,y])
z = gaussian_kde(xy)(xy)

# Sort the points by density, so that the densest points are plotted last
idx = z.argsort()
x, y, z = x[idx], y[idx], z[idx]

fig, ax = plt.subplots(1,1, figsize=(5,4))
im = ax.scatter(x, y, 
                c=z,
                alpha=0.4,
                s=10, 
                cmap='mako', 
                rasterized=True)
ax.set_title(sample)
ax.set_ylim([0,10000])
ax.set_xlim([0,3000])
ax.set_xlabel('N genes (# unique UMIs per cell)')
ax.set_ylabel('Total counts (# UMIs)')
fig.colorbar(im, pad=0, label='Density', ticks=[])


In [ ]:
sample = 'SARS-CoV-2'
sars = adata[adata.obs['batch']=='SARS2', :]

# Generate fake data
x = sars.obs['n_genes_by_counts'].to_numpy()
y = sars.obs['total_counts'].to_numpy()

# Calculate the point density
xy = np.vstack([x,y])
z = gaussian_kde(xy)(xy)

# Sort the points by density, so that the densest points are plotted last
idx = z.argsort()
x, y, z = x[idx], y[idx], z[idx]

fig, ax = plt.subplots(1,1, figsize=(5,4))
im = ax.scatter(x, y, 
                c=z,
                alpha=0.4,
                s=10, 
                cmap='mako', 
                rasterized=True)
ax.set_title(sample)
ax.set_xlabel('N genes (# unique UMIs per cell)')
ax.set_ylabel('Total counts (# UMIs)')
ax.set_ylim([0,10000])
ax.set_xlim([0,3000])
fig.colorbar(im, pad=0, label='Density', ticks=[])


In [ ]:
sample = 'SARS-CoV-2'

# Generate fake data
x = sars.obs['n_genes_by_counts'].to_numpy()
y = sars.obs['total_counts'].to_numpy()

fig, ax = plt.subplots(1,1, figsize=(5,4))
im = ax.scatter(x, y, 
                c=np.asarray(sars.raw[:, 'SARS-CoV-2'].X.todense()).flatten(),
                alpha=0.4,
                s=10, 
                cmap='RdYlBu_r', 
                rasterized=True)
ax.set_title(sample)
ax.set_xlabel('N genes (# unique UMIs per cell)')
ax.set_ylabel('Total counts (# UMIs)')
ax.set_ylim([0,10000])
ax.set_xlim([0,3000])
fig.colorbar(im, pad=0, label='SARS-CoV-2 counts',)


In [ ]:
sc.pl.pca(adata, color=['SARS-CoV-2', 'batch'])

In [ ]:
sc.external.pp.bbknn(adata, neighbors_within_batch=15, n_pcs=50)
sc.tl.umap(adata)
sc.tl.leiden(adata)
sc.pl.umap(adata, color=['SARS-CoV-2', 'batch'])

In [ ]:
sars.obs['Viral counts (raw)'] = sars.raw[:, 'SARS-CoV-2'].X.todense()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(4,3))
sns.histplot(sars.obs.loc[sars.obs['Viral counts (raw)']>0, 'Viral counts (raw)'], bins=[0,1,1e1,1e2,1e3,1e4,1e5], ax=ax)
ax.set_xscale('log')

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(4,3))
sns.distplot(sars.obs.loc[sars.obs['Viral counts (raw)']>0, 'Viral counts (raw)'], ax=ax)
# ax.set_xscale('log')

In [ ]:
sars.obs['geq10'] = 'Bystander'
sars.obs.loc[sars.obs['Viral counts (raw)']>=10, 'geq10'] = 'Infected' 

sars.obs['geq1'] = 'Bystander'
sars.obs.loc[sars.obs['Viral counts (raw)']>=1, 'geq1'] = 'Infected' 

sars.obs['geq100'] = 'Bystander'
sars.obs.loc[sars.obs['Viral counts (raw)']>=100, 'geq100'] = 'Infected' 

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(2,4))
infected_bars = [(sars.obs['geq1']=='Infected').sum()/sars.shape[0],
                 (sars.obs['geq10']=='Infected').sum()/sars.shape[0],
                 (sars.obs['geq100']=='Infected').sum()/sars.shape[0],]
bystander_bars = [(sars.obs['geq1']=='Bystander').sum()/sars.shape[0],
                  (sars.obs['geq10']=='Bystander').sum()/sars.shape[0],
                  (sars.obs['geq100']=='Bystander').sum()/sars.shape[0],]
ax.bar(['>=1', '>=10', '>=100'], infected_bars,
        color=sns.color_palette('pastel')[3], label='Infected')
ax.bar(['>=1', '>=10', '>=100'], bystander_bars,
       bottom=infected_bars,
        color=sns.color_palette('pastel')[5], label='Bystander')
ax.legend(bbox_to_anchor=(1,1))

ax.set_xlabel('Infection threshold\n(raw viral counts)')
ax.set_ylabel('Proportion of cells in SARS-CoV-2 sample')
# ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
                          

In [ ]:
sc.tl.pca(sars)
sc.pp.neighbors(sars, n_pcs=50)
sc.tl.leiden(sars)
sc.tl.umap(sars)
sc.pl.umap(sars, color=['geq10', 'SARS-CoV-2', 'leiden', 'ACE2'], ncols=2)

# Guide mapping

In [ ]:
sars.obs.index = [i.split('-SARS2')[0] for i in sars.obs.index]
sars_map.index = sars_map['cell_barcode']
sars.obs = sars.obs.merge(sars_map.loc[sars_map['num_features']==1, ['feature_call']], left_index=True, right_index=True, how='left')


In [ ]:
sc.pl.umap(sars, color=['geq10', 'SARS-CoV-2', 'leiden', 'ACE2', 'feature_call'], ncols=2)

In [ ]:
# merge both 
mock_map['cell_barcode'].isin(sars_map['cell_barcode']).sum()

In [ ]:
adata.obs

In [ ]:
mock_map.index = [i+'-MOCK' for i in mock_map['cell_barcode']]
sars_map.index = [i+'-SARS2' for i in sars_map['cell_barcode']]
mapping = pd.concat([mock_map, sars_map])
adata.obs = adata.obs.merge(mapping.loc[mapping['num_features']==1, ['feature_call']], left_index=True, right_index=True, how='left')


In [ ]:
adata.obs['raw_scv2'] = np.asarray(adata.raw[:, 'SARS-CoV-2'].X.todense()).flatten()
adata.obs['Infected'] = 'Mock'
adata.obs.loc[((adata.obs['raw_scv2']>=10) & (adata.obs['batch']=='SARS2')), 'Infected'] = 'Infected'
adata.obs.loc[((adata.obs['raw_scv2']<10) & (adata.obs['batch']=='SARS2')), 'Infected'] = 'Bystander'

In [ ]:
adata.obs['new_feature_call'] = [i.split('-')[0] if ('Targeting' not in i) else i for i in adata.obs['feature_call'].fillna('None')]

In [ ]:
if True:
    # customize list
    goi =  [
        'ACE2',
        'UBXN7',
    #     'JMJD6',
    #     'ATRX',
    #     'PCBD1',
    #     'DPF2',
        'CTSL',
    #     'PHIP',
    #     'ACVR1B',
    #     'ARID1A',
    #     'TMX3',
        'DPP4',
    #     'PIAS2',
    #     'RAD54L2',
        'ANPEP',
        'DYRK1A',
        'SMAD3',
    #     'PIAS1',
    #     'SIAH1',
    #     'ADAM17',
    #     'PHF6',
        'TMPRSS4',
    #     'SMARCE1',
    #     'CABIN1',
    #     'DOLK',
        'TMPRSS2',
        'SMARCA5',
    #     'LOC103223006',
    ]
else:
    goi = []
    for i in adata.obs.new_feature_call.unique():
        if 'Targeting' not in i and 'None' not in i:
            goi.append(i)

adata.obs['sgRNA'] = 'Other guide'
adata.obs.loc[adata.obs['feature_call'].isna(), 'sgRNA'] = 'None'
for g in goi:
    adata.obs.loc[adata.obs['new_feature_call']==g, 'sgRNA'] = g

In [ ]:
sc.pl.umap(adata[adata.obs['sgRNA']!='Other guide', :], color=['Infected', 'sgRNA'], ncols=2)

In [ ]:
cmap_guide = {v:sns.color_palette()[i] for i,v in enumerate(goi)}
cmap_guide['Other guide'] = '#DCDBE2'
cmap_guide['None'] = '#E5E5E5'

In [ ]:
adata.obs['size'] = 4
adata.obs.loc[(adata.obs['sgRNA']=='Other guide') | (adata.obs['sgRNA']=='None'), 'size'] = 1

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(4,3))
sns.scatterplot(
    x=adata.obsm['X_umap'][:,0],
    y=adata.obsm['X_umap'][:,1],
    hue=adata.obs['sgRNA'].astype(str),
    style=None,
    size=None,
    data=None,
    palette=cmap_guide,
    hue_order=None,
    hue_norm=None,
    sizes=None,
    size_order=None,
    size_norm=None,
    markers=True,
    style_order=None,
    x_bins=None,
    y_bins=None,
    units=None,
    estimator=None,
    ci=95,
    n_boot=1000,
    alpha=None,
    x_jitter=None,
    y_jitter=None,
    legend='auto',
    ax=ax,
    linewidth=0,
    s=adata.obs['size'].astype(float),
    rasterized=True,
)
ax.legend(bbox_to_anchor=(1,1))
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel('UMAP1')
ax.set_ylabel('UMAP2')
ax.set_title('Guides')
fig.savefig(os.path.join(pfp, 'umap_guides_of_interest.png'), dpi=600, bbox_inches='tight')

In [ ]:
pfp

In [ ]:
sc.settings.figdir = pfp
sc.pl.umap(adata, color=['Infected'], save='_infection.png')

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(4,3))
sns.scatterplot(
    x=adata.obsm['X_umap'][:,0],
    y=adata.obsm['X_umap'][:,1],
    hue=adata.obs['Infected'].astype(str),
    style=None,
    size=None,
    data=None,
#     palette=cmap_guide,
    hue_order=None,
    hue_norm=None,
    sizes=None,
    size_order=None,
    size_norm=None,
    markers=True,
    style_order=None,
    x_bins=None,
    y_bins=None,
    units=None,
    estimator=None,
    ci=95,
    n_boot=1000,
    alpha=None,
    x_jitter=None,
    y_jitter=None,
    legend='auto',
    ax=ax,
    linewidth=0,
    s=1, #adata.obs['size'].astype(float),
    rasterized=True,
)
ax.legend(bbox_to_anchor=(1,1))
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel('UMAP1')
ax.set_ylabel('UMAP2')
ax.set_title('Condition')
fig.savefig(os.path.join(pfp, 'umap_infection.png'), dpi=600, bbox_inches='tight')

In [ ]:
adata.obs

In [ ]:
for i in adata.obs['new_feature_call'].unique():
    print(i)
adata.obs['labels'] = [np.nan if 'None' in i or 'Non-Targ' in i else i for i in adata.obs['new_feature_call']] 

In [ ]:
for i in adata.obs['labels'].unique():
    print(i)

In [ ]:
len(adata.obs['labels'].unique())

In [ ]:
# save adata
if True :
    # save
    adata.write(os.path.join(pdfp,'crispr_shallow.h5ad'))
    print('\n... saved @'+datetime.datetime.now().strftime('%y%m%d.%H:%M:%S'))